In [1]:
import numpy as np
import tensorboard
from tensorflow import keras as K

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)  # batch_size, time steps, dimension
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # noise
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_val, y_val = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [4]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(series[:, :50], series[:, -1], test_size=0.1, shuffle=False)
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train1[:, :50], X_train1[:, -1], test_size=2 / 9, shuffle=False)

In [5]:
model = K.models.Sequential([
    K.layers.Flatten(input_shape=[n_steps, 1]),
    K.layers.Dense(1)
])

In [6]:
model.compile(
    optimizer=K.optimizers.Adam(lr=0.001),
    loss='mse'
)

C:\Users\Andrew\anaconda3\envs\TF_Developer_Certificate\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
model.fit(X_train,
          y_train,
          batch_size=64,
          validation_data=(X_val, y_val),
          epochs=20)

Epoch 1/20
110/110 [==============================] - 1s 3ms/step - loss: 0.0794 - val_loss: 0.0344
Epoch 2/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0246 - val_loss: 0.0170
Epoch 3/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0137 - val_loss: 0.0114
Epoch 4/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0098 - val_loss: 0.0090
Epoch 5/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0080 - val_loss: 0.0076
Epoch 6/20
110/110 [==============================] - 0s 3ms/step - loss: 0.0071 - val_loss: 0.0069
Epoch 7/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0065 - val_loss: 0.0065
Epoch 8/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0062 - val_loss: 0.0062
Epoch 9/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0059 - val_loss: 0.0059
Epoch 10/20
110/110 [==============================] - 0s 2ms/step - loss: 0.0057 - val_loss: 0.0057

In [23]:
print(X_train.shape)
print(y_train.shape)

(7000, 50, 1)
(7000, 1)


In [16]:
model = K.models.Sequential([
    K.layers.RNN(K.layers.LSTMCell(1), input_shape=[None, 1])
])
model.compile(
    optimizer=K.optimizers.Adam(),
    loss='mse'
)
model.fit(X_train,
          y_train,
          batch_size=64,
          epochs=2,
          validation_data=(X_val, y_val)
          )

Epoch 1/2
110/110 [==============================] - 6s 48ms/step - loss: 0.2503 - val_loss: 0.2464
Epoch 2/2
110/110 [==============================] - 5s 46ms/step - loss: 0.2287 - val_loss: 0.2247


In [ ]:
model = K.models.Sequential([
    K.layers.RNN(K.layers.LSTMCell(10), return_state=True, input_shape=[None, 1]),
    K.layers.RNN(K.layers.LSTMCell(1))
])
model.compile(
    optimizer=K.optimizers.Adam(lr=0.001),
    loss='mse'
)
model.fit(X_train,
          y_train,
          batch_size=64,
          epochs=2,
          validation_data=(X_val, y_val)
          )

In [38]:
series = generate_time_series(10000, n_steps + 10)
X_train, Y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, Y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, Y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

Y = np.empty((10000, n_steps, 10))  # each target is a sequence of 10D vectors
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]


In [39]:
print(X_train.shape)
print(Y_train.shape)

(7000, 50, 1)
(7000, 50, 10)


In [55]:
model = K.models.Sequential([
    K.layers.RNN(K.layers.GRUCell(20), return_sequences=True, input_shape=[None, 1]),
    K.layers.RNN(K.layers.GRUCell(20), return_sequences=True),
    K.layers.TimeDistributed(K.layers.Dense(10))
    # TimeDistributed: The case with Dense is that in keras from version 2.0 Dense is by default applied to only last dimension (e.g. if you apply Dense(10) to input with shape (n, m, o, p) you'll get output with shape (n, m, o, 10))
])
# model = K.models.Sequential([
#     K.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
#     K.layers.GRU(20, return_sequences=True),
#     K.layers.TimeDistributed(K.layers.Dense(10))
# ])

model.compile(
    optimizer=K.optimizers.Adam(),
    loss='mse'
)
model.fit(X_train,
          Y_train,
          batch_size=64,
          epochs=2,
          validation_data=(X_valid, Y_valid)
          )

Epoch 1/2
110/110 [==============================] - 3s 12ms/step - loss: 0.0863 - val_loss: 0.0598
Epoch 2/2
110/110 [==============================] - 1s 8ms/step - loss: 0.0555 - val_loss: 0.0517


In [60]:
model = K.models.Sequential([
    K.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                    input_shape=[None, 1]),
    K.layers.LSTM(20, return_sequences=True),
    K.layers.LSTM(20, return_sequences=True),
    K.layers.TimeDistributed(K.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, Y_valid[:, 3::2])) # (since the kernel’s size is 4, the first output of the convolutional layer will be based on the input time steps 0 to 3), and downsample the targets by a factor of 2

Epoch 1/20
219/219 [==============================] - 5s 14ms/step - loss: 0.0794 - val_loss: 0.0491
Epoch 2/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0374 - val_loss: 0.0311
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0279 - val_loss: 0.0265
Epoch 4/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0252 - val_loss: 0.0246
Epoch 5/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0235 - val_loss: 0.0232
Epoch 6/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0224 - val_loss: 0.0229
Epoch 7/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0215 - val_loss: 0.0216
Epoch 8/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0208 - val_loss: 0.0209
Epoch 9/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0203 - val_loss: 0.0205
Epoch 10/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0199 - val_lo

In [58]:
Y_train[:, 3::2].shape

(7000, 24, 10)

In [59]:
Y_train.shape

(7000, 50, 10)

In [69]:
t = np.array([[[1, 2], [1, 3], [1, 3]],
              [[3, 2], [3, 4], [1, 3]],
              [[4, 3], [3, 4], [1, 3]],
              [[5, 2], [3, 4], [1, 3]],
              [[5, 3], [4, 5], [1, 3]]])
t.shape

(5, 2, 2)

In [85]:
t[:,0::1]

array([[[1, 2],
        [1, 3]],

       [[3, 2],
        [3, 4]],

       [[4, 3],
        [3, 4]],

       [[5, 2],
        [3, 4]],

       [[5, 3],
        [4, 5]]])